In [35]:
import pandas as pd 
import datetime as dt
import os
import pymysql
import sys
import json
import numpy_financial as np
import numpy

In [2]:
data_creditos_modificar = pd.read_excel("./data/Intereses devengados 2023 Sistema vs Auditoría.xlsx", sheet_name="creditos a modificar")

data_creditos_modificar.rename(columns = {"ID DIST.": "id_distribuidor", "ID ASOCIADO":"id_asociado"}, inplace = True)

In [3]:
ids_asociados_modificar = "("+", ".join(data_creditos_modificar.id_asociado.unique().astype(str)) + ")"

In [4]:
data_creditos_modificar.columns = [k.lower().replace(" ", "_") for k in data_creditos_modificar.columns]

In [5]:
intereses_totales = lambda r, P : (r*(1+r)**16)*P/(((1+r)**16)-1)*16
num2curr = lambda x: "${:,.2f}".format(x)

In [6]:
fecha_inicio = dt.date(2023,1,1)
#fecha_inicio = dt.datetime.now().date()-dt.timedelta(days=120)

In [7]:
results_df = data_creditos_modificar.copy()

In [8]:
payments_dict = dict()

In [9]:
def get_current_intereses(results_df):
    new_dict = {"id_asociado":list(), "asociado":list(),"fecha_amortizacion":list(),
                "inicio_credito":list(), "fin_credito":list(),
                "capital":list(),
                "pago_total":list(),"capital":list(),"saldo":list(),
                "interes":list(),"iva":list(),"tasa":list(),"id_distribuidor":list(),
                #"capital_pago":list()
                "pago_acumulado":list(),"pago_esperado_acumulado":list(), #"fecha_pago":list(),
               }

    pagos_esperados_dict = dict()

    for index, row in results_df.iterrows():
        if row["id_distribuidor"] not in pagos_esperados_dict:
                pagos_esperados_dict[row["id_distribuidor"]]=0.0
        pagos_esperados_dict[row["id_distribuidor"]]+=float(row["pago_quincenal"]) + (float(row["seguro"])/16.00)

    for index, row in results_df.iterrows():
        iterative_date = row["inicio_credito"]
        tasa = (row["tasa"]/52)/100
        semanas = 16
        saldo = row["capital"]
        pct_iva = 0.16  
        #amortizacion_counter=0
        cummulative_payment=0.0
        cummulative_expected=0.0
        
        for i in range(16):
            new_dict["inicio_credito"].append(row["inicio_credito"])
            new_dict["fin_credito"].append(row["fin_credito"])
            new_dict["capital"].append(row["capital"])
            new_dict["id_distribuidor"].append(row["id_distribuidor"])
            new_dict["id_asociado"].append(row["id_asociado"])
            new_dict["asociado"].append(row["asociado"])
            fecha_amortizacion = iterative_date
            new_dict["fecha_amortizacion"].append(fecha_amortizacion)
            iterative_date = iterative_date + dt.timedelta(days=7)

            new_dict["saldo"].append(saldo)

            capital = np.ppmt(tasa,i+1,semanas,-row["capital"])
            
            interes = (intereses_totales(tasa, row["capital"]) - row["capital"]) / 16
            #new_dict["capital"].append(capital)

    
            new_dict["interes"].append(interes)

            new_dict["iva"].append(interes*pct_iva)
            pago_total = capital+interes + (interes*pct_iva)
            new_dict["pago_total"].append(pago_total)
            saldo -= capital

            new_dict["tasa"].append(row["tasa"])
            """
            cummulative_expected+=pagos_esperados_dict[row["id_distribuidor"]]
            #cummulative_payment=0.0
            if row["id_distribuidor"] in payments_dict:
                date_counter=0
                fecha_pago = ""

                for date in payments_dict[row["id_distribuidor"]][0]:
                    fecha_final=fecha_amortizacion+dt.timedelta(days=6)
                    if date_counter==0:
                        fecha_amortizacion-=dt.timedelta(days=6)
                    if fecha_amortizacion <= date <= fecha_amortizacion+dt.timedelta(days=6):
                        payment_amount=float(payments_dict[row["id_distribuidor"]][2][date_counter])
                        cummulative_payment+=payment_amount
                        fecha_pago = date
                        #print(payment_amount,date)
                    date_counter+=1
            """
            #new_dict["fecha_pago"].append(fecha_pago)
            #new_dict["capital_pago"].append(payment_amount)
            new_dict["pago_acumulado"].append(cummulative_payment)
            new_dict["pago_esperado_acumulado"].append(cummulative_expected)
            #new_dict["%_pagado"].append(cummulative_payment/cummulative_expected)
            #amortizacion_counter+=1
    processed_df = pd.DataFrame.from_dict(new_dict)

    processed_df.fecha_amortizacion = pd.to_datetime(processed_df.fecha_amortizacion)
    processed_df['year'] = processed_df['fecha_amortizacion'].dt.year
    processed_df['month'] = processed_df['fecha_amortizacion'].dt.month
    current_intereses = processed_df.query("fecha_amortizacion > '2023-01-01' and fecha_amortizacion < '2024-01-01'").groupby(['year', 'month'])['interes'].sum().reset_index()
    return current_intereses, processed_df

In [10]:
import pandas as pd

# Datos iniciales
data = {
    "fecha": ["ene-23", "feb-23", "mar-23", "abr-23", "may-23", "jun-23",
              "jul-23", "ago-23", "sep-23", "oct-23", "nov-23", "dic-23"],
    "interes": ["$2,984,275.02", "$3,018,301.71", "$3,242,862.84", "$3,352,405.54", "$3,328,462.57",
                "$3,405,379.63", "$3,203,870.78", "$3,233,416.80", "$3,259,963.52", "$3,376,434.66",
                "$3,353,729.74", "$3,321,054.63"]
}

# Crear el DataFrame
objective_intereses = pd.DataFrame(data)

# Crear un diccionario para mapear los nombres de los meses a números
meses = {
    "ene": 1, "feb": 2, "mar": 3, "abr": 4, "may": 5, "jun": 6,
    "jul": 7, "ago": 8, "sep": 9, "oct": 10, "nov": 11, "dic": 12
}

# Separar la columna 'fecha' en 'mes' y 'año'
objective_intereses['month'], objective_intereses['year'] = objective_intereses['fecha'].str.split('-', 1).str
objective_intereses['month'] = objective_intereses['month'].map(meses)
objective_intereses['year'] = '20' + objective_intereses['year']  # Añadir '20' para formar el año completo

# Convertir la columna 'interes' a valores numéricos
objective_intereses['interes'] = objective_intereses['interes'].replace('[\$,]', '', regex=True).astype(float)

# Reorganizar las columnas
objective_intereses = objective_intereses[['year', 'month', 'interes']]

# Mostrar el DataFrame resultante
objective_intereses


C:\Users\CIENTIFICO DE DATOS\AppData\Local\Temp\ipykernel_4184\875257011.py:22: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  objective_intereses['month'], objective_intereses['year'] = objective_intereses['fecha'].str.split('-', 1).str
C:\Users\CIENTIFICO DE DATOS\AppData\Local\Temp\ipykernel_4184\875257011.py:22: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  objective_intereses['month'], objective_intereses['year'] = objective_intereses['fecha'].str.split('-', 1).str


,year,month,interes
0,2023,1,2984275.02
1,2023,2,3018301.71
2,2023,3,3242862.84
3,2023,4,3352405.54
4,2023,5,3328462.57
5,2023,6,3405379.63
6,2023,7,3203870.78
7,2023,8,3233416.80
8,2023,9,3259963.52
9,2023,10,3376434.66


In [11]:
current_intereses, processed_df  = get_current_intereses(results_df)

In [12]:
print(f"Suma de lo reportado \t\t{num2curr(objective_intereses.interes.sum())}")
print(f"Suma de los actuales intereses \t{num2curr(current_intereses.interes.sum())}")
print(f"Diferencia de ambos \t\t{num2curr(sum(objective_intereses.interes - current_intereses.interes))}")

Suma de lo reportado 		$39,080,157.44
Suma de los actuales intereses 	$27,441,617.57
Diferencia de ambos 		$11,638,539.87


In [14]:
tasas_originales = results_df.groupby("id_asociado").tasa.max().reset_index()

tasas_actuales = tasas_originales.copy()

In [45]:
def obj(X):
    tasas_actuales.tasa = X
    for i, row_tasas in tasas_actuales.iterrows():
        results_df.loc[results_df.id_asociado == row_tasas['id_asociado'], 'tasa'] = row_tasas['tasa']
    current_intereses, _ = get_current_intereses(results_df)
    val = sum(abs(objective_intereses.interes - current_intereses.interes))
    print(f"{num2curr(val)}")
    return val

In [28]:
import time
from scipy.optimize import minimize
from joblib import Parallel, delayed, Memory

location = './cachedir'
memory = Memory(location, verbose=0)

# Definir una estimación inicial
x0 = tasas_actuales.tasa.values

bounds = [(75, 147) for _ in range(len(x0))]


#@memory.cache
def callback(xk):
    numpy.save(f'best_variable_{callback.iteration}.npy', np.array(xk))
    if callback.iteration % 20 == 0:
        sys.stdout.write(f"\rIteración: {callback.iteration}, f(x): {f(xk):.4f}")
    else:
        sys.stdout.write(f"\rIteración: {callback.iteration}")
    sys.stdout.flush()
    callback.iteration += 1
        
        

# Inicializar el contador de iteraciones
callback.iteration = 1
#L-BFGS-B
# Resolver el problema de optimización
solution = minimize(obj, x0, method = "L-BFGS-B",bounds=bounds, callback=callback, options={'maxiter': 2})

# Mostrar los resultados
print('Estado de optimización:', solution.success)
print('Mensaje de optimización:', solution.message)
print('Valor mínimo encontrado:', solution.fun)
print('Valores óptimos de las variables:', solution.x)


called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called
called

KeyboardInterrupt: 

In [29]:
x0

array([ 90.        ,  90.        ,  90.00000001, ..., 142.        ,
       142.        , 142.        ])

## ajuste sistemático

In [50]:
def obj_short(X):
    tasas_actuales.tasa = X
    for i, row_tasas in tasas_actuales.iterrows():
        results_df.loc[results_df.id_asociado == row_tasas['id_asociado'], 'tasa'] = row_tasas['tasa']
    current_intereses, _ = get_current_intereses(results_df)
    val = objective_intereses.interes.sum() - current_intereses.interes.sum()
    print(f"{num2curr(val)}")
    return val

In [44]:
tasas_originales = results_df.groupby("id_asociado").tasa.max().reset_index()

tasas_actuales = tasas_originales.copy()

In [48]:
x0 = tasas_actuales.tasa.values

In [54]:
step = 16
for _ in range(10):
    while True:
        x0 += step
        x0 = numpy.clip(x0, None, 147)
        valor_actual = obj_short(x0)
        if valor_actual < 0:
            x0 -= step
            step /= 2
            break

$2,523,956.12
$1,216,355.43
$1,002,634.74
$1,002,634.74


KeyboardInterrupt: 

In [55]:
x0

array([147., 147., 147., ..., 147., 147., 147.])

In [26]:
import numpy
from geneticalgorithm import geneticalgorithm as ga

def f(X):
    tasas_actuales.tasa = X
    for i, row_tasas in tasas_actuales.iterrows():
        results_df.loc[results_df.id_asociado == row_tasas['id_asociado'], 'tasa'] = row_tasas['tasa']
    current_intereses = get_current_intereses(results_df)
    val = sum(abs(objective_intereses.interes - current_intereses.interes))
    print(val)
    return val

varbound=numpy.array([[0,147]]*len(tasas_originales.tasa.values))
varbound.T[0] = numpy.load('best_variable.npy')

algorithm_param = {'max_num_iteration': 10,\
                   'population_size':5,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.2,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   
                   'max_iteration_without_improv':None}

model=ga(function=f,\
            dimension=len(tasas_originales.tasa.values),\
            variable_type='int',\
            variable_boundaries=varbound,\
            function_timeout = 10_000,
            algorithm_parameters=algorithm_param)

model.run()

17805669.962223534
17784848.9295331
17768228.39485367
17812994.928815693
17854156.940981325
|||||_____________________________________________ 10.0% GA is running...17758945.569220975
17761409.402489446
17768974.230930418
17753997.246696755
||||||||||________________________________________ 20.0% GA is running...17768218.002509702
17764899.68152033
17757952.00952873
17754893.69634087
|||||||||||||||___________________________________ 30.0% GA is running...17756169.492937297
17762557.255556136


KeyboardInterrupt: 

In [ ]:
numpy.save('best_variable.npy', model.best_variable)

In [ ]:
numpy.load('best_variable.npy')

In [ ]:
numpy.sqrt(1009469415141.319)

In [ ]:
(10_000**2)*12

In [15]:
max_pago_acumulado = processed_df.groupby("id_asociado")["pago_acumulado"].max()
max_pago_esperado = processed_df.groupby("id_asociado")["pago_esperado_acumulado"].max()
max_pago_acumulado_df = max_pago_acumulado.reset_index()
max_pago_esperado_df = max_pago_esperado.reset_index()


In [16]:
max_pago_acumulado_df.query("id_asociado=='155457'")


,id_asociado,pago_acumulado
4629,155457,62160.0


In [17]:
max_pago_esperado_df.query("id_asociado=='155457'")

,id_asociado,pago_esperado_acumulado
4629,155457,62160.0


In [18]:
for index, row in processed_df.iterrows():
    if  not row['fecha_pago']:
        #print(row["pago_acumulado"])
        #print(max_pago_acumulado_df[max_pago_acumulado_df['id_asociado'] == row["id_asociado"]]['pago_acumulado'].iloc[0])
        if row["pago_acumulado"]<= max_pago_acumulado_df[max_pago_acumulado_df['id_asociado'] == row["id_asociado"]]['pago_acumulado'].iloc[0]:
            processed_df.at[index, 'fecha_pago'] = row["fecha_amortizacion"]
            #print(row["fecha_amortizacion"])

In [19]:
processed_df.query("id_asociado=='155457'")

,id_asociado,asociado,fecha_amortizacion,pago_total,capital,saldo,interes,iva,tasa,id_distribuidor,pago_acumulado,pago_esperado_acumulado,fecha_pago,%_pagado
74912,155457,NIDIA ALEJANDRA MORALES AMADO,2023-06-27,561.141573,362.826471,6999.990000,170.96129423076925,27.353807,127.0,155456,3885.0,3885.0,2023-06-27,1.000000
74913,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-04,549.977399,371.687810,6637.163529,153.69792128706777,24.591667,127.0,155456,7770.0,7770.0,2023-07-04,1.000000
74914,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-11,539.645504,380.765570,6265.475718,136.96545992610248,21.914474,127.0,155456,11655.0,11655.0,2023-07-11,1.000000
74915,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-18,530.220845,390.065037,5884.710148,120.82397255059543,19.331836,127.0,155456,15540.0,15540.0,2023-07-18,1.000000
74916,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-25,521.782913,399.591625,5494.645111,105.33731701801503,16.853971,127.0,155456,19425.0,19425.0,2023-07-25,1.000000
74917,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-01,514.415974,409.350882,5095.053486,90.57335446672526,14.491737,127.0,155456,23310.0,23310.0,2023-08-01,1.000000
74918,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-08,508.209325,419.348491,4685.702603,76.60416791019378,12.256667,127.0,155456,27195.0,27195.0,2023-08-08,1.000000
74919,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-15,503.257570,429.590271,4266.354113,63.50629214369074,10.161007,127.0,155456,27195.0,31080.0,2023-08-15,0.875000
74920,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-22,499.660896,440.082187,3836.763842,51.360955535121434,8.217753,127.0,155456,34965.0,34965.0,2023-08-22,1.000000
74921,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-29,497.525376,450.830348,3396.681655,40.25433430020167,6.440693,127.0,155456,34965.0,38850.0,2023-08-29,0.900000


In [20]:
processed_df.to_excel("./data/amortizaciones_"+str(fecha_inicio)+"_"+str(dt.datetime.now().date())+".xlsx")  
